<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/screen_align.png" style="margin: 0 auto;">


<h1 style="text-align: center; font-size=58px;">Write Concerns</h1>

In this lesson we're going to discuss write concern, and how it can provide different levels of write durability in our application.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set.png" style="margin: 0 auto;">

Write concern is a feature of the replica set, as we'll see in a minute. Anyway, this is a little supermarket (point) application, using a replica set as its data source.

When the customer puts a food item into their cart, Mongo will send an insert statement, for that item, over to the current primary node in the replica set, and then wait for an acknowedgement back from the server.

When the primary receives this write statement, the first thing it's gonna do is perform the write in its database.

<img src="https://s3.amazonaws.com/edu-
static.mongodb.com/lessons/M220/notebook_assets/replica_set_primary_highlighted.png" style="margin: 0 auto;">

Once the write has been committed by the primary node, by default the primary will immediately send back an acknowledgement to the application that the write was committed.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_primary_secondary_highlighted_w_1.png" style="margin: 0 auto;">

So at this point, the application receives the acknowledgement (point) that it was waiting for, and it considers the write to be complete.

It assumes that the secondaries will replicate (point) the data soon, but it doesn't have any immediate proof that the secondaries did it.

### writeConcern: { w: 1 }

- Only requests an acknowledgement that **one** node applied the write
- This is the default `writeConcern` in MongoDB

So that was an example of a write with writeConcern `w: 1`. The number 1 (point) refers to the number of nodes that must perform the write statement before the client gets an acknowledgement back from the driver.

And `w: 1` is the default writeConcern in MongoDB, so if a write is sent to MongoDB without a writeConcern specified, then Mongo will assume this `w: 1` (point) setting.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_primary_highlighted.png" style="margin: 0 auto;">

So now let's consider a different level of write concern. Our shopping cart application sends a write statement to the primary node, and the primary applies that write just like it did before.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_primary_secondary_highlighted.png" style="margin: 0 auto;">

But this time, the primary waits before sending an acknowledgement back to the client. What is it waiting for, you ask? Well, let me tell you.

Before sending an acknowledgement of the write back to the client, the primary will actually wait for one of the secondary nodes to replicate the data. When the secondary applies this write, it will send an acknowledgement back to the primary, saying "hey, I applied this write to my copy of the data!"

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_primary_secondary_highlighted_w_majority.png" style="margin: 0 auto;">

So now the primary knows that it (point) applied the write, and it also knows that a secondary (point) applied the write. The primary says "ok, 2 nodes in this set have applied the write," and then send an acknowledgement back to the client.

As denoted here (point), this write was completed with `w: majority`. `w: majority` is another way of saying that we want a majority of nodes in the replica set to apply the write to their data before the client gets an acknowledgement back from the driver. In this case, we had a three-node replica set, and two nodes makes a majority. So we sent acknowledgement after two nodes applied this write.

Think of `w: majority` as a kind of contract with the client that this write will not be lost, even in the event of hosts going down. If our application sends a write with `w: majority` and gets back an acknowledgement of the write, we know that a majority of nodes have applied the write. In the case of a 3 nodes replica set, it would require 2 nodes performing the write. In the case of a 5 nodes replica set, it would require 3 nodes to perform the write.

The current primary could actually go down immediately after we get an acknowledgement, and we could still rest assured that another node has captured the write we sent.

### writeConcern: { w: 'majority' }

- Requests acknowledgement that a **majority of nodes** in the replica set applied the write
- Takes longer than `w: 1`
- Is more durable than `w: 1`
    - Useful for ensuring vital writes are majority-committed

So with `w: majority` the connection is going to wait for a majority (point) of nodes to apply a write before sending an acknowledgement back to the client. For that reason, it takes a little longer, and is subject to replication lag. But there is no additional load on the primary, so the primary can still perform the same number of writes per second.

However, `w: majority` essentially guarantees to the client that a write will not be rolled back during failover, because the write was committed to a majority of nodes.

This is useful when some of our application's writes are vital to the success of the application. A common example of this is a new user on a website. These types of operations must succeed, because without an account the user cannot do anything on the site.

### writeConcern: { w: 0 }

- Does **not** request an acknowledgement that any nodes applied the write
    - "Fire-and-forget"
- Fastest writeConcern level
- Least durable writeConcern

So I just wanna discuss one more writeConcern, `w: 0`. By now you must have realized that when the value of `w` is a number, it's the number of nodes that must apply a write before the client receieves an acknowledgement.

We can pass any value to this `w` field, but obviously `w` can only be as high as the total number of nodes in our replica set.

Following that rule, when `w` is 0, none of the nodes actually need to apply a write before the client receives an acknowledgement of that write. This means that when we're using `w: 0`, there is a chance we get an acknowledgement before any data has been written. So if the server crashes, we might lose some writes.

This type of write is referred to as a "fire-and-forget" operation, because it sends the write and doesn't really worry about the response. But this isn't entirely true, because the acknowledgement from a `w: 0` write can also alert us to network errors and socket exceptions, so the client can implement some logic to figure out if a write was actually received by the database.

Anyway, writing with `w: 0` is very fast and can be useful for less important writes that occur frequently. For example, if an Internet of Things device is sending a ping to Mongo every two minutes to report its status, it might be ok to speed up each write operation at the risk of losing a few writes.

## Summary

- `w: 1` is the default writeConcern, and it makes sure writes have been committed by at least 1 node
- `w: majority` ensures that writes are committed by a majority of nodes
    - Slower, but very durable
- `w: 0` does not ensure that a write was committed by any nodes
    - Very fast, but less durable


So to recap, `w: 1` is the default writeConcern, and it commits a write to 1 node before sending an acknowledgement to the client.

`w: majority` commits a write to a majority of nodes in the replica set before sending an acknowledgement to the client. The application will have to wait a little longer for a response, however it should not have a performance impact as long as you have enough connections to the Primary to handle all your requests.

`w: 0` does not commit the write at all, but sends an acknowledgment back to the client immediately. So there's a higher chance that we lose data in the event of a primary going down.